In [7]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import requests
import matplotlib.pyplot as plt
import joblib
%matplotlib inline
from bs4 import BeautifulSoup
from gensim import models
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# ***Importing dataset into a dataframe***


In [8]:
dataset = pd.read_csv('final.csv')
pos = pd.read_csv('Positive_Dictionary.csv')
neg = pd.read_csv('Negative_Dictionary.csv')

In [9]:
corpus = []
for i in range(0,len(dataset.News) ):
  review = re.sub('[^a-zA-Z]', ' ', dataset['News'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)
print("Corpus\n\n",corpus)

Corpus

 ['trade setup mt k immedi support nifti use move exit high beta stock', 'tech view nifti slip consolid phase upsid hurdl', 'sensex end margin lower choppi trade nifti hold', 'explain chart nifti went x year', 'icici lombard gener insur compani ltd share nifti drop', 'berger paint india ltd share nifti drop', 'torrent pharmaceut ltd share fall nifti drop', 'share price petronet lng ltd fall nifti weaken', 'share price tv motor compani ltd fall nifti weaken', 'share adani green energi ltd fall nifti drop', 'crompton greav consum electr ltd share nifti gain', 'mrf ltd share rise nifti gain', 'indian oil corpor ltd share nifti drop', 'pidilit industri ltd share fall nifti drop', 'share price shriram transport financ compani ltd rise nifti weaken', 'bandhan bank ltd share fall nifti drop', 'share jubil foodwork ltd rise nifti drop', 'feder bank ltd share fall nifti drop', 'share lic hous financ ltd fall nifti drop', 'share price pi industri ltd fall nifti weaken', 'share siemen ltd

# ***ANN(Traning)***

In [10]:
#X=m,y=n
cv = CountVectorizer(max_features = 5000)
count_vect = cv.fit(corpus)
#Pickle count vectorizer
with open('cntvect.pickle', 'wb') as f:
    pickle.dump(count_vect, f)
#Creating matrix X and y
m = cv.fit_transform(corpus).toarray()
n = dataset.iloc[:, -1].values
m_train, m_test, n_train, n_test = train_test_split(m, n, test_size = 0.20, random_state = 0)
ann = Sequential()
ann.add(Dense(units=19, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=19, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=19, activation='relu'))
ann.add(Dropout(0.5))
ann.add(Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
ann.fit(m_train, n_train, batch_size = 32, epochs = 10, validation_data=(m_test,n_test))
n_pred = ann.predict(m_test)
n_pred = (n_pred > 0.5)
print(np.concatenate((n_pred.reshape(len(n_pred),1), n_test.reshape(len(n_test),1)),1))
cm = confusion_matrix(n_test, n_pred)
print(cm)
ann_accuracy = accuracy_score(n_test, n_pred)
print(ann_accuracy*100)
print(ann.summary())




Epoch 1/10
90/90 [==============================] - 2s 5ms/step - loss: 0.6629 - accuracy: 0.5969 - val_loss: 0.6118 - val_accuracy: 0.6078
Epoch 2/10
90/90 [==============================] - 0s 5ms/step - loss: 0.5558 - accuracy: 0.6401 - val_loss: 0.4305 - val_accuracy: 0.8428
Epoch 3/10
90/90 [==============================] - 0s 3ms/step - loss: 0.4106 - accuracy: 0.8246 - val_loss: 0.2726 - val_accuracy: 0.9152
Epoch 4/10
90/90 [==============================] - 0s 3ms/step - loss: 0.2921 - accuracy: 0.9036 - val_loss: 0.2070 - val_accuracy: 0.9263
Epoch 5/10
90/90 [==============================] - 0s 4ms/step - loss: 0.2406 - accuracy: 0.9238 - val_loss: 0.1716 - val_accuracy: 0.9346
Epoch 6/10
90/90 [==============================] - 0s 3ms/step - loss: 0.1864 - accuracy: 0.9412 - val_loss: 0.1544 - val_accuracy: 0.9346
Epoch 7/10
90/90 [==============================] - 0s 3ms/step - loss: 0.1617 - accuracy: 0.9478 - val_loss: 0.1535 - val_accuracy: 0.9346
Epoch 8/10
90/90 [==

# ***News extraction using BS4***

In [11]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }

def get_news():
  news= []
  time_stamp= []
  for j in range(1,5):
      url = 'https://economictimes.indiatimes.com/lazyloadlistnew.cms?msid=2146843&curpg='+str(j)
      r = requests.get(url,{'headers':headers})
      soup = BeautifulSoup(r.text, 'html.parser')
      for i in range(0,10):
           headlines = soup.find_all('div',{'class': 'eachStory'})[i].find('h3').text
           time = soup.find_all('div',{'class': 'eachStory'})[i].find('time').text
           news.append(headlines)
           time_stamp.append(time)
  return (news, time_stamp)

news, time_stamp = get_news()
data_set = pd.DataFrame(data= news, columns=['News'])  
data_time= pd.DataFrame(data=time_stamp, columns=['Time'])

In [12]:
ann_pred= []
news, time = get_news()
data_set = pd.DataFrame(data= news, columns=['News'])
for i in range(0, len(data_set.News)):
  new_news = data_set.News[i]
  new_news = re.sub('[^a-zA-Z]', ' ', new_news)
  new_news = new_news.lower()
  new_news = new_news.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  new_news = [ps.stem(word) for word in new_news if not word in set(all_stopwords)]
  new_news = ' '.join(new_news)
  new_corpus = [new_news]
  new_m_test = cv.transform(new_corpus).toarray()
  new_n_pred = ann.predict(new_m_test)
  ann_pred.append(new_n_pred)
  if new_n_pred > 0.5:
    new_n_pred = 1
  else:
    new_n_pred = 0
ann_pred
for i in range(0,len(ann_pred)):
  if ann_pred[i] > 0.5:
    ann_pred[i]= 1
  else:
    ann_pred[i]=0


# ***Dictionary Based Approach (LSB)***

In [13]:
def preprocess(): 
  news1, time= get_news()
  stop_words= set(stopwords.words("english"))
  j=0
  noofpos1 = list()
  noofneg1 = list()
  p,n=0,0
  for i in news1:
      filter_sent1= []
      p, n=0,0
      for j in word_tokenize(i):
          if j not in stop_words:
              filter_sent1.append(j)
              for k in pos['0']:
                if j == k:
                  p += 1
              for k in neg['0']:
                if j == k:
                  n += 1
              
      noofpos1.append(p)
      noofneg1.append(n)
  
  sent_total = []
  news_sent = []
  for i,j in zip(noofpos1,noofneg1):
    sent_total.append(i-j)
  
  for i in sent_total:
    if(int(i)==0):
      news_sent.append(0)
    elif (int(i)>0):
      news_sent.append(1)
    elif (int(i)<0):
      news_sent.append(0)
  return news_sent
  
 
dic_based1= preprocess() 
dic_based = pd.DataFrame(data = dic_based1, columns=['Dictionary'])

# ***Classification models***

In [14]:
models= []
model_names = ['LinearSVC','LogisticRegression','RandomForestClassifier']
# model_names = ['LinearSVC','LogisticRegression','MLPClassifier','SGDClassifier','AdaBoostClassifier','RandomForestClassifier','XGBClassifier','MultinomialNB','BernoulliNB']
for i in range(0,3):
  model= joblib.load(model_names[i])
  models.append(model)
print(models)
news_data = pd.read_csv('final.csv')
X= news_data['News']
y= news_data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit(X_train)
test_dataset = countvector.transform(news)
all_model_pred = []
for model in models:
    y_pred = model.predict(test_dataset)
    all_model_pred.append(y_pred)

[LinearSVC(), LogisticRegression(), RandomForestClassifier()]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 1.0.2. This might lead to breaking code or invalid 

# ***Dataframe concat***

In [15]:
predictions_ann = pd.DataFrame(data= ann_pred)
predictions_ml = pd.DataFrame(data= all_model_pred)
predictions_ml = predictions_ml.transpose()
dic_based.columns = ['Dictionary']
predictions_ml.columns=['LinearSVC','LogisticRegression','RandomForestClassifier']
predictions_ann.columns = ['ANN']
prediction = pd.concat([data_set, dic_based], axis=1)
predictions = pd.concat([prediction, predictions_ml], axis=1)
predictions = pd.concat([predictions, predictions_ann], axis=1 )
predictions

,News,Dictionary,LinearSVC,LogisticRegression,RandomForestClassifier,ANN
0,Technical indicators hint at high volatility i...,0,1.0,1.0,1.0,0
1,Sebi nod means distressed asset funds can buy ...,0,0.0,0.0,1.0,0
2,Bharti's surprise fundraising talk sends stock...,0,1.0,1.0,1.0,1
3,Stock market to remain closed today on Republi...,0,0.0,0.0,1.0,1
4,"A two-year rally, and still only 50 of top 100...",1,1.0,1.0,1.0,1
5,Swaminathan View: A big global stock market cr...,0,1.0,1.0,1.0,0
6,"LIC reports net profit of Rs 1,437 crore for H...",0,1.0,1.0,1.0,0
7,Wall Street slides over 2% as tech rout deepen...,0,1.0,1.0,1.0,1
8,Nikhil Kamath on why he is skeptical about new...,0,1.0,1.0,1.0,1
9,Sebi notifies stricter norms for appointment o...,0,1.0,1.0,1.0,0


In [16]:
group_1= []
flag=0
for i in range(0, len(predictions.News)):
  var= predictions.iloc[i]
  g1=var[2:3]
  for j in g1.values:
    if j == 0:
      group_1.append(0)
    else:
      group_1.append(1)

In [17]:
group_2= []
flag=0
for i in range(0, len(predictions.News)):
  var= predictions.iloc[i]
  g2=var[3:4]
  for j in g2.values:
    if j == 0:
      group_2.append(0)
    else:
      group_2.append(1)


In [18]:
group_3= []
flag=0
for i in range(0, len(predictions.News)):
  var= predictions.iloc[i]
  g3=var[4:5]
  for j in g3.values:
    if j == 0:
      group_3.append(0)
    else:
      group_3.append(1)


In [19]:
# g1_call = pd.DataFrame(data= group_1, columns=['Group 1'])
# g2_call = pd.DataFrame(data= group_2, columns=['Group 2'])
# g3_call = pd.DataFrame(data= group_3, columns=['Group 3'])
g1_call = pd.DataFrame(data= group_1, columns=['LinearSVC'])
g2_call = pd.DataFrame(data= group_2, columns=['LogisticRegression'])
g3_call = pd.DataFrame(data= group_3, columns=['RandomForestClassifier'])
Frame= pd.concat([data_set, data_time],axis=1)
Frame0 = pd.concat([Frame, predictions_ann],axis=1)
Frame1 = pd.concat([Frame0, g1_call],axis=1)
Frame2 = pd.concat([Frame1, g2_call],axis=1)
Frame3 = pd.concat([Frame2, g3_call],axis=1)
Final_bits= pd.concat([Frame3,dic_based], axis=1)
#Final_bits

# ***Final output***

In [20]:
pd.set_option("display.max_colwidth", 0)
Final_bits

,News,Time,ANN,LinearSVC,LogisticRegression,RandomForestClassifier,Dictionary
0,Technical indicators hint at high volatility in near term,"Jan 26, 2022, 10:29 AM IST",0,1,1,1,0
1,Sebi nod means distressed asset funds can buy bad loans directly from banks,"Jan 26, 2022, 10:14 AM IST",0,0,0,1,0
2,Bharti's surprise fundraising talk sends stock up 4%,"Jan 26, 2022, 09:59 AM IST",1,1,1,1,0
3,Stock market to remain closed today on Republic Day,"Jan 26, 2022, 09:47 AM IST",1,0,0,1,0
4,"A two-year rally, and still only 50 of top 100 cos beat the Nifty","Jan 26, 2022, 09:07 AM IST",1,1,1,1,1
5,Swaminathan View: A big global stock market crash is coming,"Jan 25, 2022, 11:25 PM IST",0,1,1,1,0
6,"LIC reports net profit of Rs 1,437 crore for H1FY22","Jan 25, 2022, 09:44 PM IST",0,1,1,1,0
7,Wall Street slides over 2% as tech rout deepens; Fed meeting eyed,"Jan 25, 2022, 08:20 PM IST",1,1,1,1,0
8,Nikhil Kamath on why he is skeptical about new tech cos and his expectations from Budget 2022,"Jan 25, 2022, 08:11 PM IST",1,1,1,1,0
9,Sebi notifies stricter norms for appointment of MD at listed cos,"Jan 25, 2022, 06:27 PM IST",0,1,1,1,0


In [21]:
bit_calls = pd.read_csv('bits_and_calls.csv')
import ast
bit= []
bits= []
compare_bits= []
for x in range(0,len(bit_calls)):
  bit.append(bit_calls['Bit pattern'][x])
for x in bit:
  bits.append(ast.literal_eval(x))
bits
for x in (bits):
  compare_bits.append(''.join(x))
gen_list= []
for i in range(0, len(Final_bits.News)):
  var= Final_bits.iloc[i]
  bit=var[2:].values  
  gen_list.append(''.join(map(str,bit)))
call= []
for x in gen_list:
  for i in range(0,len(compare_bits)):
    if x == compare_bits[i]:
      call.append(bit_calls['Calls'][i])
calls = pd.DataFrame(data= call, columns=['Calls'])
Final_calls= pd.concat([Final_bits,calls], axis=1)

news_calls=[]
todays_news = []

Final_calls['Time']= pd.to_datetime(Final_calls['Time'])
Final_calls['Date']=Final_calls['Time'].dt.day
for i in range(0,len(Final_calls)):
  if (Final_calls['Date'][i]== Final_calls['Date'][0]):
    news=str(Final_calls['News'][i])
    today_news_call= str(Final_calls['Calls'][i])
    if (today_news_call== 'strong_buy'):
      today_news_call = 1
    elif (today_news_call== 'buy'):
        today_news_call = 1
    elif (today_news_call== 'hold'):
        today_news_call = 0
    elif (today_news_call== 'sell'):
        today_news_call = 0
    else:
        today_news_call = 0
    news_calls.append(today_news_call)
    todays_news.append(news)

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [22]:
pd.set_option("display.max_colwidth", 0)
Final_calls.drop("Date", axis = 1)

,News,Time,ANN,LinearSVC,LogisticRegression,RandomForestClassifier,Dictionary,Calls
0,Technical indicators hint at high volatility in near term,2022-01-26 10:29:00,0,1,1,1,0,hold
1,Sebi nod means distressed asset funds can buy bad loans directly from banks,2022-01-26 10:14:00,0,0,0,1,0,strong_sell
2,Bharti's surprise fundraising talk sends stock up 4%,2022-01-26 09:59:00,1,1,1,1,0,strong_buy
3,Stock market to remain closed today on Republic Day,2022-01-26 09:47:00,1,0,0,1,0,hold
4,"A two-year rally, and still only 50 of top 100 cos beat the Nifty",2022-01-26 09:07:00,1,1,1,1,1,strong_buy
5,Swaminathan View: A big global stock market crash is coming,2022-01-25 23:25:00,0,1,1,1,0,hold
6,"LIC reports net profit of Rs 1,437 crore for H1FY22",2022-01-25 21:44:00,0,1,1,1,0,hold
7,Wall Street slides over 2% as tech rout deepens; Fed meeting eyed,2022-01-25 20:20:00,1,1,1,1,0,strong_buy
8,Nikhil Kamath on why he is skeptical about new tech cos and his expectations from Budget 2022,2022-01-25 20:11:00,1,1,1,1,0,strong_buy
9,Sebi notifies stricter norms for appointment of MD at listed cos,2022-01-25 18:27:00,0,1,1,1,0,hold
